## Using ffmpeg-python to trim videos and audio files

In this module, we will trim videos and audio files using ffmpeg-python package.

### Case senario
**Case 1**: Imagine that you have annotated a video for co-speech manual gestures and speech. Next, you want to extract a video for each annotated gesture. How do you do it?

**Case 2**: You have 60 videos from an experiment. The video starts 3-5 minutes before the trials start, and you want to trim the part before the trials. How do you do it?
<br>
<br>

Obviously, manually trimming the videos isn't the most efficient option. But I have good news for you: **You can use Python to automate this tedious task!!**

### Playing with ffmpeg-python

In [2]:
import ffmpeg

media_start = 10 # seconds (time at which you want video to start)
input_file_path = "input/salma_hayek_video.mp4"
trimmed_video_file_path = "output/salma_hayek_video_trimmed.mp4"

ffmpeg.input(input_file_path, ss=media_start).output(trimmed_video_file_path, c='copy').run() #this function runs the file form inoput and gives desired vidoe in output folder 

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

(None, None)

## Case 1

In [3]:
### Import libraries
import os
import pandas as pd
from datetime import datetime, timedelta
import ffmpeg

In [4]:
### Folders
media_folder = "02_media" #where the video is
presentation_folder = "03_presentation_logs" #timestamps of trial info
processed_folder = os.path.join(presentation_folder, "02_processed") #trial and round information
trials_folder = os.path.join(processed_folder, "02_trials") #info on trials

### Functions

In [ ]:
def trim_video(folder, processed_folder, media_start):
    # Get the video files
    video_files = [f for f in os.listdir(folder) if f.endswith('.mp4')] #a list of paths to all video files if it ends with mp4 
    # Trim the video files
    for video_file in video_files:
        video_file_path = os.path.join(folder, video_file)
        trimmed_video_file_path = os.path.join(processed_folder, video_file)

        if not os.path.exists(trimmed_video_file_path):
            print("Trimming video file: " + video_file)
            ffmpeg.input(video_file_path, ss=media_start).output(trimmed_video_file_path, c='copy').run()


def trim_audio(folder, processed_folder, media_start):
    # Get the audio files
    audio_files = [f for f in os.listdir(folder) if f.endswith('.wav')]
    # Trim the audio files
    for audio_file in audio_files:
        audio_file_path = os.path.join(folder, audio_file)
        trimmed_audio_file_path = os.path.join(processed_folder, audio_file)

        if not os.path.exists(trimmed_audio_file_path):
            print("Trimming audio file: " + audio_file)
            ffmpeg.input(audio_file_path, ss=media_start).output(trimmed_audio_file_path, c='copy').run()

In [ ]:
for file in os.listdir(trials_folder):
    media_file_folder = os.path.join(media_folder, file.split("_")[0]) # for eacg file in trial folder, take the media folder and split the trial name and use first part
    processed_folder = os.path.join(media_file_folder, "processed")
    
    if file.endswith("_trials.csv"):
        task_start, media_start = calculate_media_trim(file)
        
        #if the media_start is less than 1 second, skip the file
        if task_start < "00:00:01.000":
            continue
        
        make_processed_folder(processed_folder)
        trim_video(media_file_folder, processed_folder, media_start)
        trim_audio(media_file_folder, processed_folder, media_start)